In [ ]:
import pulp

In [ ]:

# Define the problem
evacuation_problem = pulp.LpProblem("Evacuation_Optimization", pulp.LpMinimize)

# Define the variables
# Flow variables f_ij for every edge in the graph
flow_vars = pulp.LpVariable.dicts("flow", (nodes, nodes), lowBound=0, cat='Continuous')

# Binary variables x_k for every safety channel k
channel_vars = pulp.LpVariable.dicts("channel", safety_channels, cat='Binary')

# Assume corner_points, exits, classroom_exits, edges_dict are defined per your image outline
# ...

# Define the objective function
objective_function = pulp.lpSum([evacuation_time[i] for i in individuals])
evacuation_problem += objective_function, "Total Evacuation Time"

# Flow capacity constraint
for i, j in edges:
    evacuation_problem += flow_vars[i][j] <= capacity(i, j, width_ij), f"Flow_Capacity_Constraint_{i}_{j}"

# Flow conservation constraint
for node in nodes:
    if node not in (source, sink):
        evacuation_problem += (
            pulp.lpSum([flow_vars[i][node] for i in nodes if (i, node) in edges]) ==
            pulp.lpSum([flow_vars[node][j] for j in nodes if (node, j) in edges]),
            f"Flow_Conservation_{node}"
        )

# Non-negativity and binary constraints are defined implicitly by variable definitions

# Define the capacity as a function of width for each path
def capacity(i, j, width_ij):
    # Implement the function to calculate capacity based on width and other factors
    return calculated_capacity




Solve:

In [ ]:
# Solve the problem
evacuation_problem.solve()

# Check if a valid solution was found
if pulp.LpStatus[evacuation_problem.status] == 'Optimal':
    # Print out the flows and channel states
    print("Optimal flows:")
    for i, j in edges:
        print(f"Flow from {i} to {j}: {flow_vars[i][j].value()}")

    print("\nChannel states:")
    for k in safety_channels:
        print(f"Channel {k} state: {('Open' if channel_vars[k].value() == 1 else 'Closed')}")

    # You can also calculate and print the total evacuation time if needed
    # ...
else:
    print("No optimal solution found.")